# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
Cities_df = pd.read_csv("../output_data/cities.csv")
Cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
locations = Cities_df[["Lat", "Lng"]].astype(float)
humidity = Cities_df["Humidity"].astype(int)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
perfect_df = Cities_df.loc[(Cities_df['Max Temp'] > 70) & (Cities_df['Max Temp'] < 80) & (Cities_df['Cloudiness'] < 10)]
perfect_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotels = []
Lat = perfect_df['Lat'].tolist()
Lng = perfect_df['Lng'].tolist()

for city in range(len(Lat)):

    lat = Lat[city]
    lng = Lng[city]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
perfect_df["Nearest Hotel"] = hotels
perfect_df

C:\Users\austi\AppData\Local\Temp/ipykernel_18036/3279852355.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfect_df["Nearest Hotel"] = hotels


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Nearest Hotel
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,,Opuwo
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,,Pisco
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,,Capitólio
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,,Ambilobe
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,,CIDREIRA
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,,Koumac
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,,Sao Filipe
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,,Morondava
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,,Jagalbet
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,,Nador


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)
fig

ValueError: too many values to unpack (expected 2)